In [1]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:

import os
os.chdir("..")
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/datasource_hackathon


In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data_folder = "data"
price_data = pd.read_csv(f"{data_folder}/Phase 0 - Price.csv")
sales_data = pd.read_csv(f"{data_folder}/Phase 0 - Sales.csv")
inventory_data = pd.read_csv(f"{data_folder}/Phase 0 - Inventory.csv")

In [5]:
# Let's merge the three datasets (price, sales, inventory) on common columns: 'Client', 'Warehouse', and 'Product'
# These columns are shared across the datasets, and we can merge them based on this common information.

# First, we need to reshape the data from wide format (with date columns) to long format for easier merging
price_data_melted = price_data.melt(id_vars=['Client', 'Warehouse', 'Product'], var_name='Date', value_name='Price')
sales_data_melted = sales_data.melt(id_vars=['Client', 'Warehouse', 'Product'], var_name='Date', value_name='Sales')
inventory_data_melted = inventory_data.melt(id_vars=['Client', 'Warehouse', 'Product'], var_name='Date', value_name='Inventory')

# Merge the datasets on 'Client', 'Warehouse', 'Product', and 'Date'
merged_data = pd.merge(price_data_melted, sales_data_melted, on=['Client', 'Warehouse', 'Product', 'Date'], how='outer')
merged_data = pd.merge(merged_data, inventory_data_melted, on=['Client', 'Warehouse', 'Product', 'Date'], how='outer')

# Convert 'Date' to datetime for easier time-based analysis
merged_data['Date'] = pd.to_datetime(merged_data['Date'])


In [6]:
merged_data = merged_data.sort_values(by=['Client', 'Warehouse', 'Product', 'Date'])

In [7]:
merged_data.head()

,Client,Warehouse,Product,Date,Price,Sales,Inventory
0,0,1,367,2020-07-06,10.900001,7.0,0.0
1,0,1,367,2020-07-13,10.900001,7.0,0.0
2,0,1,367,2020-07-20,10.900001,7.0,0.0
3,0,1,367,2020-07-27,15.582857,7.0,0.0
4,0,1,367,2020-08-03,27.289999,7.0,0.0


In [8]:
merged_data = merged_data.drop(columns=['Client'],axis=1)

In [9]:
merged_data["id"] = (
    merged_data["Warehouse"].astype("str") + "_" + merged_data["Product"].astype("str")
)

In [10]:
merged_data = merged_data.groupby(["id", "Date"], as_index=False).agg(
    {"Price": "mean", "Sales": "sum", "Inventory": "sum"}
)

In [11]:
merged_data.sort_values(by=["id", "Date"], inplace=True)

In [12]:
id_mindt_df = (
    merged_data.loc[merged_data["Price"].notnull()]
    .groupby("id", as_index=False)["Date"]
    .min()
)

In [13]:
id_mindt_df = id_mindt_df.rename(columns={"Date": "MinDate"})

In [14]:
id_mindt_df.head()

,id,MinDate
0,0_10052,2022-06-20
1,0_10097,2022-11-21
2,0_10357,2023-09-25
3,0_10645,2023-05-01
4,0_10921,2022-06-20


In [22]:
merged_data_new = pd.merge(merged_data, id_mindt_df, on=["id"], how="left")

In [23]:
merged_data_new.shape

(2559010, 6)

In [24]:
merged_data_new = merged_data_new.loc[merged_data_new["Date"] >= merged_data_new["MinDate"]].reset_index(drop=True)
print(merged_data_new.shape)

(1419748, 6)


#### id-Date min

In [25]:
combo_df = merged_data_new.groupby("id", as_index=False).agg({"Date": ["min", "max", "nunique"]})

In [26]:
# ravel the hierarchical columns into a single level
combo_df.columns = ["_".join(x) for x in combo_df.columns.ravel()]

In [27]:
combo_df.head()

,id_,Date_min,Date_max,Date_nunique
0,0_10052,2022-06-20,2023-10-02,68
1,0_10097,2022-11-21,2023-10-02,46
2,0_10357,2023-09-25,2023-10-02,2
3,0_10645,2023-05-01,2023-10-02,23
4,0_10921,2022-06-20,2023-10-02,68


In [28]:
merged_data_new = merged_data_new.sort_values(by=["id", "Date"]).reset_index(drop=True)

In [30]:
merged_data_new.head()

,id,Date,Price,Sales,Inventory,MinDate
0,0_10052,2022-06-20,97.00500,2.0,0.0,2022-06-20
1,0_10052,2022-06-27,95.69333,3.0,0.0,2022-06-20
2,0_10052,2022-07-04,95.43000,2.0,0.0,2022-06-20
3,0_10052,2022-07-11,94.33000,4.0,0.0,2022-06-20
4,0_10052,2022-07-18,62.98167,6.0,0.0,2022-06-20


In [ ]:
## Next Steps: ID level (warehouse + product)
# 1. Id level make the data continous weekly - every id should have all the weeks starting from the min date to max date
# 2. Fill missing values with 0/mean (Price , sales , inventory)
# 3. For every id we have to add 13 weeks of artificial index
# 4. For inventory we need to have some different approach to be used with markers columns
# 5. For price and sales have lag and rolling mean features starting from 13 weeks target
# 6. Seperate out the combinations which have less than 52 weeks of data - A set will be go for regression model and B set will go recursive modelling

